In [17]:
from pathlib import Path

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')
csv_files = []
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)
csv_files

[PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.5/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_DSC_DAall_epoch_200/no_TTA/overlap_0.5/th_0.5/BCE_DSC_DAall_epoch_200_no_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_DAsafe_epoch_40/no_TTA/overlap_0.5/th_0.5/Focal_Tversky_DAsafe_epoch_40_no_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_DAsafe_epoch_150/TTA/overlap_0.5/th_0.4/Focal_Tversky_DAsafe_epoch_150_TTA_overlap_0.5_th_0.4.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_DAsafe_epoch_150/TTA/overlap_0.5/th_0.5/Focal_Tversky_DAsafe_epoch_150_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home

In [31]:
import pandas as pd
import glob
import os

rows_list = []
cols_to_remove = ['tif_paths', 'id', 'pred_paths']

for file in csv_files:
    filename = os.path.basename(file)
    df = pd.read_csv(file)
    
    if len(df) >= 80:
        mean_row = df.iloc[[79]].copy()
        clean_row = mean_row.drop(columns=cols_to_remove, errors='ignore')
        
        # --- PARSING FILENAME ---
        clean_row['Losses'] = filename.split('_DA')[0] 
        clean_row['epoch'] = filename.split('epoch_')[-1].split('_')[0] 
        clean_row['Train DA'] = filename.split('_DA')[-1].split('_')[0] 
        clean_row['tta_status'] = 'no_TTA' if 'no_TTA' in filename else 'TTA'
        
        if 'overlap_' in filename:
            overlap_val = filename.split('overlap_')[1].split('_')[0]
            clean_row['overlap'] = float(overlap_val)
            
        if 'th_' in filename:
            th_val = filename.split('th_')[1].replace('.csv', '')
            clean_row['threshold'] = float(th_val)
        
        rows_list.append(clean_row)

# 2. Create the new combined DataFrame
combined_df = pd.concat(rows_list, ignore_index=True)

# 3. REORDER COLUMNS: Define the leading order
# We list exactly what we want first, then add the rest of the columns
fixed_cols = ['Losses', 'epoch', 'Train DA', 'tta_status', 'overlap', 'threshold']
other_cols = [c for c in combined_df.columns if c not in fixed_cols]

combined_df = combined_df[fixed_cols + other_cols]

# 4. View the result
combined_df

,Losses,epoch,Train DA,tta_status,overlap,threshold,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge
0,BCE_DSC,80,safe,no_TTA,0.5,0.5,0.516074,0.130427,0.819026,0.543678,1.335766,1.513423
1,BCE_DSC,200,all,no_TTA,0.5,0.5,0.485808,0.109025,0.770171,0.524402,1.475593,1.594309
2,Focal_Tversky,40,safe,no_TTA,0.5,0.5,0.519876,0.130271,0.823788,0.549909,1.304752,1.463048
3,Focal_Tversky,150,safe,TTA,0.5,0.4,0.537427,0.186253,0.812687,0.563173,1.178807,1.439129
4,Focal_Tversky,150,safe,TTA,0.5,0.5,0.531597,0.176505,0.817828,0.549730,1.333118,1.444002
5,Focal_Tversky,150,safe,TTA,0.5,0.6,0.508585,0.144277,0.785108,0.544325,1.291426,1.544986
